# ABS Monthly International Trade 5368

## Python set-up

In [1]:
# system imports
import sys

# analytic imports
import pandas as pd
import matplotlib.pyplot as plt

# local imports
from abs_data_capture import (
    get_fs_constants,
    get_plot_constants,
    get_meta_constants,
    get_ABS_meta_and_data,
    get_identifier,
    clear_cache,
)
from plotting import (
    finalise_plot,
    recalibrate,
    clear_chart_dir,
    set_chart_dir,
)

# pandas display settings
pd.options.display.max_rows = None
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 999

# plotting stuff
plt.style.use("fivethirtyeight")

In [2]:
# in case of emergency
if False:
    clear_cache()

## Get data from ABS

In [3]:
catalogue_id = "5368"
source, CHART_DIR, META_DATA = get_fs_constants(catalogue_id)
abs_trade = get_ABS_meta_and_data(catalogue_id, verbose=False)
if abs_trade is None:
    sys.exit(-1)
meta_trade = abs_trade[META_DATA]
RECENT, plot_times, plot_tags = get_plot_constants(meta_trade)
did_col, id_col, table_col, type_col, unit_col = get_meta_constants()

Found URL for a ZIP file on ABS web page
Retrieving zip-file from cache ...
Extracting DataFrames from the zip-file ...
Caution: Could not find the "Index" sheet in 5368092.xlsx


## Plot

### Initialisation


In [4]:
clear_chart_dir(CHART_DIR)
set_chart_dir(CHART_DIR)

### Headline charts

In [5]:
table = "1"
headline = meta_trade[
    (meta_trade[table_col] == table)
][did_col].unique()

for h in headline:
    # secure the data
    f = pd.DataFrame()
    for service_type in "Seasonally Adjusted", "Trend":
        id, units = get_identifier(meta_trade, h, service_type, table)
        f[service_type] = abs_trade[table][id]
    f, units = recalibrate(f, units, verbose=True)
        
    # plot
    for start, tag in zip(plot_times, plot_tags):
        subset = f[f.index >= pd.Period(start, freq="M")] if start else f
        ax = None
        for (
            col,
            color,
            lw,
        ) in zip(subset.columns, ["darkblue", "darkorange"], [1, 3]):
            ax = subset[col].plot(color=color, lw=lw, ax=ax)
        ax.legend(loc="best")
        title = (
            h
            .replace('Credits', 'Credits (Exports)')
            .replace('Debits', 'Debits (Imports)')
            .replace(' ;', '')
            .strip()
        )
        lfooter = "Australia. Current Prices. "
        if f['Trend'].isna().iloc[-1]:
            last = f['Trend'].last_valid_index()
            lfooter += f'No trand data after {last}. '
        finalise_plot(
            ax,
            title=title,
            ylabel=units,
            tag=tag,
            rfooter=f"{source} {table}",
            lfooter=lfooter,
            # show=True,
        )

## Finished

In [6]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Mon Mar 06 2023 15:57:57

Python implementation: CPython
Python version       : 3.11.0
IPython version      : 8.11.0

matplotlib: 3.7.0
sys       : 3.11.0 | packaged by conda-forge | (main, Jan 15 2023, 05:44:48) [Clang 14.0.6 ]
pandas    : 1.5.3

Watermark: 2.3.1



In [7]:
print("Finished")

Finished
